In [1]:
import findspark
findspark.init()

In [2]:
import json,os

In [3]:
# from pyspark_ai import SparkAI

# spark_ai = SparkAI()
# spark_ai.activate()  # active partial functions for Spark DataFrame

In [4]:
import pyspark
from pyspark.sql import functions as F

In [5]:
import logging

logger = logging.getLogger(__name__)

In [6]:
os.getcwd()

'/Users/akshayranade/Documents/Personal/pyspark_code_generator/notebooks'

In [7]:
os.chdir('../transform')

In [8]:
from code_master import CodeMaster

In [9]:
from pyspark.sql import SparkSession, DataFrame
spark = SparkSession.builder.appName('abc').getOrCreate()

24/08/07 14:10:17 WARN Utils: Your hostname, Akshays-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.10.129 instead (on interface en0)
24/08/07 14:10:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 14:10:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
input_data={}

In [11]:
simpleData = [("James", "Sales", 3000),
        ("Michael", "Sales", 4600),
        ("Robert", "Sales", 4100),
        ("Maria", "Finance", 3000),
        ("James", "Sales", 3000),
        ("Scott", "Finance", 3300),
        ("Jen", "Finance", 3900),
        ("Jeff", "Marketing", 3000),
        ("Kumar", "Marketing", 2000),
        ("Saif", "Sales", 4100)
]
schema = ["employee_name", "department", "salary"]

employee_department = spark.createDataFrame(data=simpleData, schema = schema)

#Add to input_data dictionary
input_data['employee_department'] = employee_department

simpleData = [("James", "Stewart", 11111),
    ("Michael", "Paine", 44444),
    ("Robert", "Baggins", 12345),
    ("Maria", "Hanks", 34567),
    ("James", "Snape", 43215),
    ("Scott", "Potter", 65234),
    ("Jen", "Granger", 76234),
    ("Jeff", "Weasley", 54234),
    ("Kumar", "Oppenheimer", 34567),
    ("Saif", "Tesla", 43215)
]
schema = ["first_name", "last_name", "pin_code"]

employee_demo = spark.createDataFrame(data=simpleData, schema = schema)

#Add to input_data dictionary
input_data['employee_demo'] = employee_demo

In [12]:
employee_department.show(20)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [13]:
for column_name in employee_department.columns:
    employee_department=employee_department.withColumnRenamed(column_name,column_name+"_left")

In [14]:
employee_department.show(20)

+------------------+---------------+-----------+
|employee_name_left|department_left|salary_left|
+------------------+---------------+-----------+
|             James|          Sales|       3000|
|           Michael|          Sales|       4600|
|            Robert|          Sales|       4100|
|             Maria|        Finance|       3000|
|             James|          Sales|       3000|
|             Scott|        Finance|       3300|
|               Jen|        Finance|       3900|
|              Jeff|      Marketing|       3000|
|             Kumar|      Marketing|       2000|
|              Saif|          Sales|       4100|
+------------------+---------------+-----------+



In [15]:
for column_name in employee_department.columns:
    employee_department=employee_department.withColumnRenamed(column_name,column_name.replace("_left",""))

In [16]:
employee_department.show(20)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [17]:
# xyz.write.parquet('/Users/akshayranade/Documents/Personal/spark_output/xyz')

In [18]:
# auto sales data from https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand
# data = [('Toyota', 1849751, -9), ('Ford', 1767439, -2), ('Chevrolet', 1502389, 6),
#         ('Honda', 881201, -33), ('Hyundai', 724265, -2), ('Kia', 693549, -1),
#         ('Jeep', 684612, -12), ('Nissan', 682731, -25), ('Subaru', 556581, -5),
#         ('Ram Trucks', 545194, -16), ('GMC', 517649, 7), ('Mercedes-Benz', 350949, 7),
#         ('BMW', 332388, -1), ('Volkswagen', 301069, -20), ('Mazda', 294908, -11),
#         ('Lexus', 258704, -15), ('Dodge', 190793, -12), ('Audi', 186875, -5),
#         ('Cadillac', 134726, 14), ('Chrysler', 112713, -2), ('Buick', 103519, -42),
#         ('Acura', 102306, -35), ('Volvo', 102038, -16), ('Mitsubishi', 102037, -16),
#         ('Lincoln', 83486, -4), ('Porsche', 70065, 0), ('Genesis', 56410, 14),
#         ('INFINITI', 46619, -20), ('MINI', 29504, -1), ('Alfa Romeo', 12845, -30),
#         ('Maserati', 6413, -10), ('Bentley', 3975, 0), ('Lamborghini', 3134, 3),
#         ('Fiat', 915, -61), ('McLaren', 840, -35), ('Rolls-Royce', 460, 7)]

# dq = spark.createDataFrame(data, ["Brand", "US_Sales_2022", "Sales_Change_Percentage"])

In [19]:
# dq.show(20)

In [20]:
col_name = 'col1_left'

In [21]:
col_name.rstrip("_left")

'col1'

In [22]:
# data = {}

In [23]:
# data['xyz'] = xyz
# data['dq'] = dq

In [24]:
input_data

{'employee_department': DataFrame[employee_name: string, department: string, salary: bigint],
 'employee_demo': DataFrame[first_name: string, last_name: string, pin_code: bigint]}

In [25]:
f = open ('../conf/pipeline.json', "r")
pipeline_dict = json.loads(f.read())

In [26]:
pipeline_dict

{'module_name': 'ar',
 'version': '0.1',
 'pipeline': {'stage_1': {'join': {'left_table': 'employee_department',
    'right_table': 'employee_demo',
    'join_condition': ['employee_name_left == first_name'],
    'join_type': 'left',
    'output': 'employee_all'}},
  'stage_2': {'aggregate': {'table_name': 'employee_all',
    'group_by_cols': ['department'],
    'aggregate_functions': {'salary': 'sum', 'employee_name': 'count'},
    'output': 'stats_by_department'}},
  'stage_3': {'aggregate': {'table_name': 'employee_all',
    'group_by_cols': ['pin_code'],
    'aggregate_functions': {'salary': 'sum'},
    'output': 'income_by_region'}}}}

In [27]:
#code master

code_master_obj = CodeMaster(pipeline=pipeline_dict, data=input_data, spark=spark)

In [28]:
df, df_dict = code_master_obj.build_pipeline()

In [29]:
df_dict

{'employee_all': DataFrame[employee_name: string, department: string, salary: bigint, first_name: string, last_name: string, pin_code: bigint],
 'stats_by_department': DataFrame[department: string, count_employee_name: bigint, sum_salary: bigint],
 'income_by_region': DataFrame[pin_code: bigint, sum_salary: bigint]}

In [30]:
df_dict['employee_all'].show(20)

+-------------+----------+------+----------+-----------+--------+
|employee_name|department|salary|first_name|  last_name|pin_code|
+-------------+----------+------+----------+-----------+--------+
|        James|     Sales|  3000|     James|      Snape|   43215|
|        James|     Sales|  3000|     James|    Stewart|   11111|
|      Michael|     Sales|  4600|   Michael|      Paine|   44444|
|       Robert|     Sales|  4100|    Robert|    Baggins|   12345|
|        James|     Sales|  3000|     James|      Snape|   43215|
|        James|     Sales|  3000|     James|    Stewart|   11111|
|        Maria|   Finance|  3000|     Maria|      Hanks|   34567|
|        Scott|   Finance|  3300|     Scott|     Potter|   65234|
|          Jen|   Finance|  3900|       Jen|    Granger|   76234|
|         Jeff| Marketing|  3000|      Jeff|    Weasley|   54234|
|        Kumar| Marketing|  2000|     Kumar|Oppenheimer|   34567|
|         Saif|     Sales|  4100|      Saif|      Tesla|   43215|
+---------

In [31]:
df_dict['stats_by_department'].show(20)

+----------+-------------------+----------+
|department|count_employee_name|sum_salary|
+----------+-------------------+----------+
|     Sales|                  7|     24800|
|   Finance|                  3|     10200|
| Marketing|                  2|      5000|
+----------+-------------------+----------+



In [32]:
df_dict['income_by_region'].show(20)

+--------+----------+
|pin_code|sum_salary|
+--------+----------+
|   65234|      3300|
|   54234|      3000|
|   12345|      4100|
|   43215|     10100|
|   11111|      6000|
|   76234|      3900|
|   44444|      4600|
|   34567|      5000|
+--------+----------+



In [16]:
pipeline = pipeline_dict['pipeline']

In [20]:
for stage in pipeline.keys():
    print (stage)
    print(pipeline.get(stage))
    stage_operations = pipeline.get(stage)
    for oper in stage_operations.keys():
        print(oper)
        print(stage_operations.get(oper))

stage_1
{'select': {'table_name': 'xyz', 'cols_to_select': ['a', 'c', 'v', 'p'], 'drop': ['p'], 'dedup': 'True'}}
select
{'table_name': 'xyz', 'cols_to_select': ['a', 'c', 'v', 'p'], 'drop': ['p'], 'dedup': 'True'}
stage_2
{'select': {'table_name': 'dq'}}
select
{'table_name': 'dq'}


In [64]:
sel_details = pipeline['pipeline']['stage_1']['select']
sel_details

{'table_name': 'xyz',
 'cols_to_select': ['a', 'c', 'v', 'p'],
 'drop': ['p'],
 'dedup': 'True'}

In [65]:
df1 = selection(sel_details)

In [1]:
df1.show(20)

NameError: name 'df1' is not defined

In [63]:
def selection(details:dict):
    try:
        df_sel = data[details["table_name"]]
    except:
        logger.error("No dataframe mentioned to select.")
        exit()

    if 'cols_to_select' in details.keys():
        try:
            df_sel = df_sel.select(*details["cols_to_select"])
        except:
            logger.info('Column selection %s for table %s is incorrect.', str(details["cols_to_select"]), details["table_name"])
            exit()
    
    if 'drop' in details.keys():
        df_sel = df_sel.drop(*details["drop"])

    if 'dedup' in details.keys():
        if details['dedup'].upper() == "TRUE":
            df_sel = df_sel.drop_duplicates()

    return df_sel